<a href="https://colab.research.google.com/github/adipai/data-decent/blob/main/src/experiments/ambari_vuln_sampling_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip data.zip

In [ ]:
!pip install sdv

In [3]:
!pip install DataSynthesizer

In [4]:
# All imports here
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns
import random
import time
from scipy.io import arff
from sdv.datasets.local import load_csvs
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer
from DataSynthesizer.DataDescriber import DataDescriber
from DataSynthesizer.DataGenerator import DataGenerator
from DataSynthesizer.lib.utils import display_bayesian_network
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import RandomOverSampler
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix

## Data preprocessing

In [5]:
def preprocess_data_train(X_train):
    # Count missing values before handling missing data
    missing_before = np.isnan(X_train).sum()

    # Handle missing data
    imputer = SimpleImputer(strategy='mean')
    X_train = imputer.fit_transform(X_train)

    # Count missing values after handling missing data
    missing_after = np.isnan(X_train).sum()

    # Normalize numeric columns
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)

    return X_train, scaler, imputer

def preprocess_data_test(X_test, scaler, imputer):
    # Count missing values before handling missing data
    missing_before = np.isnan(X_test).sum()

    # Handle missing data
    X_test = imputer.transform(X_test)

    # Count missing values after handling missing data
    missing_after = np.isnan(X_test).sum()

    # Normalize numeric columns
    X_test = scaler.transform(X_test)

    return X_test

## Experiments

### Dataset : JavaScript Vulnerbalities

In [6]:
train_data_path = f"data/Bug_Reports/ambari-train.csv"
test_data_path = f"data/Bug_Reports/ambari-test.csv"
train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)
X_train = train_df.iloc[:, :-1]
y_train = train_df.iloc[:, -1]
X_test = test_df.iloc[:, :-1]
y_test = test_df.iloc[:, -1]

## Training and testing using ML models

In [7]:
# Generic function to test synthetic data using LR, SVM, DT

def evaluate_models(X_train, X_test, y_train, y_test, random_state=42):

    # Initialize classifiers
    classifiers = {
        "Logistic Regression": LogisticRegression(random_state=random_state),
        "SVM": SVC(random_state=random_state),
        "Decision Tree": DecisionTreeClassifier(random_state=random_state)
    }

    # Results dictionary to store evaluation metrics
    results = {}

    # Iterate over classifiers
    for name, clf in classifiers.items():
        # Fit classifier
        clf.fit(X_train, y_train)

        # Predictions
        y_pred = clf.predict(X_test)

        # Evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # AUC-ROC
        if hasattr(clf, "predict_proba"):
            y_prob = clf.predict_proba(X_test)[:,1]
        else:
            y_prob = clf.decision_function(X_test)
        fpr, tpr, thresholds = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)

        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)

        # Store results
        results[name] = {
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1,
            "ROC AUC": roc_auc,
            "Confusion Matrix": cm
        }

        # Plot AUC-ROC curve
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'{name} - AUC-ROC Curve')
        plt.legend(loc='lower right')
        plt.savefig(f'{name}_auc_roc_curve.png', dpi=300)
        plt.close()

        # Plot confusion matrix
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'{name} - Confusion Matrix')
        plt.savefig(f'{name}_confusion_matrix.png', dpi=300)
        plt.close()

    return results

In [8]:
results = evaluate_models(X_train, X_test, y_train, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
print(results)

{'Logistic Regression': {'Accuracy': 0.978, 'Precision': 0.25, 'Recall': 0.2857142857142857, 'F1 Score': 0.26666666666666666, 'ROC AUC': 0.755722978846711, 'Confusion Matrix': array([[487,   6],
       [  5,   2]])}, 'SVM': {'Accuracy': 0.986, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.6734279918864097, 'Confusion Matrix': array([[493,   0],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.968, 'Precision': 0.09090909090909091, 'Recall': 0.14285714285714285, 'F1 Score': 0.1111111111111111, 'ROC AUC': 0.5612865835989568, 'Confusion Matrix': array([[483,  10],
       [  6,   1]])}}


# SDV - Oversampling

In [10]:
def do_sdv(X_train, y_train):
  train_df = pd.concat([X_train, y_train], axis=1)
  class_counts = y_train.value_counts()

  # Find minority class label
  minority_class_label = class_counts.idxmin()

  # Filter rows with minority class label
  minority_df = train_df[train_df.iloc[:, -1] == minority_class_label]

  # Calculate counts of majority and minority classes
  majority_count = class_counts.max()
  minority_count = class_counts.min()

  metadata_data = SingleTableMetadata()
  metadata_data.detect_from_dataframe(minority_df)
  print(metadata_data)
  # Generate synthetic data using GaussianCopulaSynthesizer
  synthesizer_breast_data = GaussianCopulaSynthesizer(metadata_data)
  synthesizer_breast_data.fit(minority_df)

  # Print sample synthetic data
  synthesizer_breast_data.reset_sampling()
  sd1 = synthesizer_breast_data.sample(num_rows=majority_count-minority_count)
  return sd1, train_df

# Function to add synthetic data to the main DataFrame based on percentage
def add_synthetic_data(main_df, synthetic_df, percentage, seed=42):
    # Calculate number of rows to sample
    num_rows = int(len(synthetic_df) * percentage)

    # Sample the specified percentage of synthetic data
    sampled_synthetic_data = synthetic_df.sample(n=num_rows, replace=False, random_state=seed)
    # print(sampled_synthetic_data)

    # Concatenate sampled synthetic data with main DataFrame
    combined_df = pd.concat([main_df, sampled_synthetic_data], ignore_index=True)
    # print(combined_df)
    return combined_df

# Random Over-Sampling

In [11]:
def find_minority_data(X, y):
    labels, counts = np.unique(y, return_counts=True)
    min_label = min(zip(counts, labels))[1]
    indices_with_min_label = np.where(y == min_label)[0]
    X_min, y_min = X[indices_with_min_label], y[indices_with_min_label]

    # Other class samples
    indices_without_min_label = np.where(y != min_label)[0]
    X_remaining, y_remaining = X[indices_without_min_label], y[indices_without_min_label]

    return X_min, y_min, X_remaining, y_remaining, min_label

def random_oversampling(X_train, y_train, oversampling_ratios, seed=42):

  oversampled_X_train_ratios = dict()
  oversampled_y_train_ratios = dict()
  X_minority, y_minority, X_remaining, y_remaining, min_label = find_minority_data(X_train, y_train)
  ideal_samps = len(X_remaining) - len(X_minority)

  oversampling_samps = [int(ideal_samps * (oversampling_ratio)) for oversampling_ratio in oversampling_ratios]
  for oversampling_samp, oversampling_ratio in zip(oversampling_samps, oversampling_ratios):

    sampling_strategy = {min_label: len(X_minority) + oversampling_samp}
    X_train_upsampled, y_train_upsampled = RandomOverSampler(sampling_strategy=sampling_strategy, random_state = seed).fit_resample(X_train, y_train)

    oversampled_X_train_ratios[oversampling_ratio] = X_train_upsampled
    oversampled_y_train_ratios[oversampling_ratio] = y_train_upsampled

  return list(oversampled_X_train_ratios.values()), list(oversampled_y_train_ratios.values())


# SMOTE Over-Sampling

In [12]:
def find_minority_data(X, y):
    labels, counts = np.unique(y, return_counts=True)
    min_label = min(zip(counts, labels))[1]
    indices_with_min_label = np.where(y == min_label)[0]
    X_min, y_min = X[indices_with_min_label], y[indices_with_min_label]

    # Other class samples
    indices_without_min_label = np.where(y != min_label)[0]
    X_remaining, y_remaining = X[indices_without_min_label], y[indices_without_min_label]

    return X_min, y_min, X_remaining, y_remaining, min_label

def smote_oversampling(X_train, y_train, oversampling_ratios, seed=42):

  oversampled_X_train_ratios = dict()
  oversampled_y_train_ratios = dict()
  X_minority, y_minority, X_remaining, y_remaining, min_label = find_minority_data(X_train, y_train)
  ideal_samps = len(X_remaining) - len(X_minority)

  oversampling_samps = [int(ideal_samps * (oversampling_ratio)) for oversampling_ratio in oversampling_ratios]
  for oversampling_samp, oversampling_ratio in zip(oversampling_samps, oversampling_ratios):

    sampling_strategy = {min_label: len(X_minority) + oversampling_samp}
    X_train_upsampled, y_train_upsampled = SMOTE(sampling_strategy=sampling_strategy, random_state = seed).fit_resample(X_train, y_train)

    oversampled_X_train_ratios[oversampling_ratio] = X_train_upsampled
    oversampled_y_train_ratios[oversampling_ratio] = y_train_upsampled

  return list(oversampled_X_train_ratios.values()), list(oversampled_y_train_ratios.values())


# SVM-SMOTE Over-Sampling

In [13]:
def find_minority_data(X, y):
    labels, counts = np.unique(y, return_counts=True)
    min_label = min(zip(counts, labels))[1]
    indices_with_min_label = np.where(y == min_label)[0]
    X_min, y_min = X[indices_with_min_label], y[indices_with_min_label]

    # Other class samples
    indices_without_min_label = np.where(y != min_label)[0]
    X_remaining, y_remaining = X[indices_without_min_label], y[indices_without_min_label]

    return X_min, y_min, X_remaining, y_remaining, min_label

def svm_smote_oversampling(X_train, y_train, oversampling_ratios, seed=42):

  oversampled_X_train_ratios = dict()
  oversampled_y_train_ratios = dict()
  X_minority, y_minority, X_remaining, y_remaining, min_label = find_minority_data(X_train, y_train)
  ideal_samps = len(X_remaining) - len(X_minority)

  oversampling_samps = [int(ideal_samps * (oversampling_ratio)) for oversampling_ratio in oversampling_ratios]
  for oversampling_samp, oversampling_ratio in zip(oversampling_samps, oversampling_ratios):

    sampling_strategy = {min_label: len(X_minority) + oversampling_samp}
    X_train_upsampled, y_train_upsampled = SVMSMOTE(sampling_strategy=sampling_strategy, random_state = seed).fit_resample(X_train, y_train)

    oversampled_X_train_ratios[oversampling_ratio] = X_train_upsampled
    oversampled_y_train_ratios[oversampling_ratio] = y_train_upsampled

  return list(oversampled_X_train_ratios.values()), list(oversampled_y_train_ratios.values())


# Intelligent Pruning

In [14]:
def find_majority_data(X, y):
    labels, counts = np.unique(y, return_counts=True)
    max_label = max(zip(counts, labels))[1]
    indices_with_max_label = np.where(y == max_label)[0]
    X_maj, y_maj = X[indices_with_max_label], y[indices_with_max_label]

    # Exclude majority class samples
    indices_without_max_label = np.where(y != max_label)[0]
    X_remaining, y_remaining = X[indices_without_max_label], y[indices_without_max_label]

    return X_maj, y_maj, X_remaining, y_remaining, min(counts)

def do_clustering(X, y, labels):
  clustered_X = defaultdict(list)
  clustered_y = defaultdict(list)

  for i, label in enumerate(labels):
      clustered_X[label].append(X[i])
      clustered_y[label].append(y[i])

  # Sort clustered_X and clustered_y in descending order based on the length of values in each dictionary
  sorted_clustered_X = dict(sorted(clustered_X.items(), key=lambda x: -len(x[1])))
  sorted_clustered_y = dict(sorted(clustered_y.items(), key=lambda x: -len(x[1])))

  return sorted_clustered_X, sorted_clustered_y


def intelligent_prune_data(pruning_samps, pruning_ratios, clustered_X, clustered_y, per_cluster_pruning_ratio=0.7, seed=42):
  random.seed(seed)
  pruning_ratios_X_maj, pruning_ratios_y_maj = defaultdict(list), defaultdict(list)
  for pruning_samp, pruning_ratio in zip(pruning_samps, pruning_ratios):
    samps = 0
    # print("For Pruning samps: ", pruning_samp)
    prune_samps = pruning_samp
    # print(prune_samps)
    clustered_X_new = defaultdict(list)
    clustered_y_new = defaultdict(list)
    # Iterate over the sorted dictionaries
    for label, values_X in clustered_X.items():
        # Calculate the number of samples to prune
        num_samples_to_prune = int(prune_samps * per_cluster_pruning_ratio)
        if(num_samples_to_prune > len(values_X)):
          num_samples_to_prune = len(values_X)//2
          prune_samps -= num_samples_to_prune
        else:
          prune_samps -= num_samples_to_prune

        # Randomly choose samples to prune
        indices_to_prune = random.sample(range(len(values_X)), num_samples_to_prune)

        # Prune the samples from clustered_X and clustered_y
        clustered_X_new[label] = [values_X[i] for i in range(len(values_X)) if i not in indices_to_prune]
        clustered_y_new[label] = [clustered_y[label][i] for i in range(len(clustered_y[label])) if i not in indices_to_prune]

    iter = 0
    while(prune_samps > 0):
        if(iter>=100):
          break
        for label, values_X in clustered_X_new.items():
          if(prune_samps <=0 or len(values_X) <= 0):
            break
          # print(len(values_X))
          index_to_prune = random.sample(range(len(values_X)), 1)
          clustered_X_new[label] = [values_X[i] for i in range(len(values_X)) if i not in index_to_prune]
          clustered_y_new[label] = [clustered_y_new[label][i] for i in range(len(clustered_y_new[label])) if i not in index_to_prune]

          prune_samps -= 1
        iter += 1

    for label in clustered_X_new:
        pruning_ratios_X_maj[pruning_ratio].extend(clustered_X_new[label])
        pruning_ratios_y_maj[pruning_ratio].extend(clustered_y_new[label])

  return pruning_ratios_X_maj, pruning_ratios_y_maj

def combine_data(pruning_ratios, pruning_ratios_X_maj, pruning_ratios_y_maj, X_remaining, y_remaining):

  pruning_ratios_X, pruning_ratios_y = defaultdict(list), defaultdict(list)
  for pruning_ratio in pruning_ratios:
    pruning_ratios_X[pruning_ratio].extend(pruning_ratios_X_maj[pruning_ratio])
    pruning_ratios_X[pruning_ratio].extend(X_remaining)

    pruning_ratios_y[pruning_ratio].extend(pruning_ratios_y_maj[pruning_ratio])
    pruning_ratios_y[pruning_ratio].extend(y_remaining)

  return pruning_ratios_X, pruning_ratios_y

def do_intelligent_pruning(X, y, ratio, per_cluster_pruning_ratio=0.7, seed=42):

  X_maj, y_maj, X_remaining, y_remaining, min_class_samples = find_majority_data(X, y)
  kmeans = KMeans(n_clusters=3, random_state = 42)
  kmeans.fit(X_maj)
  labels = kmeans.labels_
  clustered_X, clustered_y = do_clustering(X_maj, y_maj, labels)

  pruning_best = len(X_maj)-min_class_samples
  pruning_samps = [int(pruning_best * ratio)]
  pruning_ratios = [ratio]

  pruning_ratios_X_maj, pruning_ratios_y_maj = intelligent_prune_data(pruning_samps, pruning_ratios, clustered_X, clustered_y, \
                                                                      per_cluster_pruning_ratio=per_cluster_pruning_ratio, seed=seed)

  pruning_ratios_X, pruning_ratios_y = combine_data(pruning_ratios, pruning_ratios_X_maj, pruning_ratios_y_maj, X_remaining, y_remaining)

  return list(pruning_ratios_X.values()), list(pruning_ratios_y.values())

# Random Pruning

In [15]:
"""
inputs:
X: np.array
y: np.array
percentage: from 0% upto 100%, enter int value
"""
def random_prune_data(X, y, ratio, seed = 42):
  # preprocessed_X, scaler, imputer = preprocess_data_train(X)
  # preprocessed_X_test = preprocess_data_test(X_test, scaler, imputer)

  # X_train, y_train = preprocessed_X_train.to_numpy(), y_train.to_numpy()
  # X_test, y_test = preprocessed_X_test.to_numpy(), y_test.to_numpy()
  np.random.seed(seed)
  labels_count = {}
  labels = np.unique(y)
  for label in labels:
    labels_count[label] = np.count_nonzero(y == label)
  max_label = min_label = labels[0]
  for label in labels_count:
    if labels_count[label] > labels_count[max_label]:
      max_label = label
    if labels_count[label] < labels_count[min_label]:
      min_label = label

  # print("Max", max_label, labels_count[max_label])
  # print("Min", min_label, labels_count[min_label])

  prune_counts = {}
  prune_indexes = {}
  for label in labels_count:
    prune_counts[label] = labels_count[label] - labels_count[min_label]
    prune_indexes[label] = np.where(y == label)[0]

  prune_amount = int(ratio * sum(map(lambda x: x[1], prune_counts.items())))
  prune_it = {}

  while prune_amount > 0:
    for label in labels:
      if (len(prune_indexes[label]) - labels_count[min_label]) > 0 and prune_amount > 0:
        random_index = np.random.choice(len(prune_indexes[label]))
        random_item = prune_indexes[label][random_index]
        prune_indexes[label] = np.delete(prune_indexes[label], random_index)
        if prune_it.get(label, None) is None:
          prune_it[label] = np.array([])
        prune_it[label] = np.append(prune_it[label], [random_item])
        prune_amount -= 1



  formatted_indexes = np.array([])
  for label in prune_indexes:
    formatted_indexes = np.append(formatted_indexes, prune_indexes[label])
  formatted_indexes = np.sort(formatted_indexes)
  new_arr = np.array([np.int64(i) for i in formatted_indexes])

  return X[new_arr], y[new_arr]

In [16]:
ratios = [ratio for ratio in np.arange(0.2, 1.1, 0.2)]

# Calling Intelligent Pruning

In [17]:
results_intelligent_pruning = dict()
per_cluster_pruning_ratios = [0.5, 0.7, 0.9, 1]

for per_cluster_pruning_ratio in per_cluster_pruning_ratios:
  print(f'For per-cluster pruning ratio {per_cluster_pruning_ratio}')
  for ratio in ratios:
    X_train_copy, y_train_copy = X_train.copy(), y_train.copy()

    intelligent_pruned_X_train, intelligent_pruned_y_train = do_intelligent_pruning(X_train_copy.to_numpy(), y_train_copy.to_numpy(), ratio, per_cluster_pruning_ratio=per_cluster_pruning_ratio)

    preprocessed_intelligent_pruned_X_train, scaler, imputer = preprocess_data_train((np.array(intelligent_pruned_X_train))[0])
    preprocessed_X_test = preprocess_data_test(X_test, scaler, imputer)

    intelligent_pruned_X_train, intelligent_pruned_y_train = preprocessed_intelligent_pruned_X_train, (np.array(intelligent_pruned_y_train))[0]
    intelligent_pruned_X_test, intelligent_pruned_y_test = preprocessed_X_test, y_test.to_numpy()
    print(f"Train data pruned intelligently at {ratio * 100}% :")
    results = evaluate_models(intelligent_pruned_X_train, intelligent_pruned_X_test, intelligent_pruned_y_train, intelligent_pruned_y_test)
    print(results)
    results_intelligent_pruning[ratio] = results
    print("_______________________________________________________________________________")

For per-cluster pruning ratio 0.5


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data pruned intelligently at 20.0% :


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Logistic Regression': {'Accuracy': 0.934, 'Precision': 0.03571428571428571, 'Recall': 0.14285714285714285, 'F1 Score': 0.05714285714285715, 'ROC AUC': 0.5818603303390322, 'Confusion Matrix': array([[466,  27],
       [  6,   1]])}, 'SVM': {'Accuracy': 0.986, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.4555201390901188, 'Confusion Matrix': array([[493,   0],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.964, 'Precision': 0.07692307692307693, 'Recall': 0.14285714285714285, 'F1 Score': 0.1, 'ROC AUC': 0.5592581860330339, 'Confusion Matrix': array([[481,  12],
       [  6,   1]])}}
_______________________________________________________________________________


...

{'Logistic Regression': {'Accuracy': 0.932, 'Precision': 0.034482758620689655, 'Recall': 0.14285714285714285, 'F1 Score': 0.05555555555555555, 'ROC AUC': 0.6354679802955665, 'Confusion Matrix': array([[465,  28],
       [  6,   1]])}, 'SVM': {'Accuracy': 0.986, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.43465662126919735, 'Confusion Matrix': array([[493,   0],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.958, 'Precision': 0.18181818181818182, 'Recall': 0.5714285714285714, 'F1 Score': 0.27586206896551724, 'ROC AUC': 0.7674587076209795, 'Confusion Matrix': array([[475,  18],
       [  3,   4]])}}
_______________________________________________________________________________


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data pruned intelligently at 80.0% :
{'Logistic Regression': {'Accuracy': 0.936, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.5838887279049552, 'Confusion Matrix': array([[468,  25],
       [  7,   0]])}, 'SVM': {'Accuracy': 0.88, 'Precision': 0.01818181818181818, 'Recall': 0.14285714285714285, 'F1 Score': 0.032258064516129024, 'ROC AUC': 0.4149521877716604, 'Confusion Matrix': array([[439,  54],
       [  6,   1]])}, 'Decision Tree': {'Accuracy': 0.952, 'Precision': 0.16, 'Recall': 0.5714285714285714, 'F1 Score': 0.25, 'ROC AUC': 0.764416111272095, 'Confusion Matrix': array([[472,  21],
       [  3,   4]])}}
_______________________________________________________________________________


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data pruned intelligently at 100.0% :
{'Logistic Regression': {'Accuracy': 0.848, 'Precision': 0.014084507042253521, 'Recall': 0.14285714285714285, 'F1 Score': 0.025641025641025637, 'ROC AUC': 0.6186612576064909, 'Confusion Matrix': array([[423,  70],
       [  6,   1]])}, 'SVM': {'Accuracy': 0.544, 'Precision': 0.008888888888888889, 'Recall': 0.2857142857142857, 'F1 Score': 0.017241379310344827, 'ROC AUC': 0.3819182845552014, 'Confusion Matrix': array([[270, 223],
       [  5,   2]])}, 'Decision Tree': {'Accuracy': 0.736, 'Precision': 0.03007518796992481, 'Recall': 0.5714285714285714, 'F1 Score': 0.05714285714285714, 'ROC AUC': 0.6548826427122573, 'Confusion Matrix': array([[364, 129],
       [  3,   4]])}}
_______________________________________________________________________________


#Calling Random Pruning

In [18]:
results_random_pruning = dict()
for ratio in ratios:
  random_pruned_X_train, random_pruned_y_train = random_prune_data(X_train.to_numpy(), y_train.to_numpy(), ratio)
  preprocessed_random_pruned_X_train, scaler, imputer = preprocess_data_train(random_pruned_X_train)
  preprocessed_X_test = preprocess_data_test(X_test, scaler, imputer)

  random_pruned_X_train, random_pruned_y_train = preprocessed_random_pruned_X_train, random_pruned_y_train
  random_pruned_X_test, random_pruned_y_test = preprocessed_X_test, y_test.to_numpy()

  print(f"Train data pruned randomly at {ratio * 100}% :")
  results = evaluate_models(random_pruned_X_train, random_pruned_X_test, random_pruned_y_train, random_pruned_y_test)
  print(results)
  results_random_pruning[ratio] = results
  print("_______________________________________________________________________________")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data pruned randomly at 20.0% :


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Logistic Regression': {'Accuracy': 0.952, 'Precision': 0.05263157894736842, 'Recall': 0.14285714285714285, 'F1 Score': 0.07692307692307693, 'ROC AUC': 0.6111272095044914, 'Confusion Matrix': array([[475,  18],
       [  6,   1]])}, 'SVM': {'Accuracy': 0.986, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.45523036800927263, 'Confusion Matrix': array([[493,   0],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.97, 'Precision': 0.1, 'Recall': 0.14285714285714285, 'F1 Score': 0.11764705882352941, 'ROC AUC': 0.5623007823819183, 'Confusion Matrix': array([[484,   9],
       [  6,   1]])}}
_______________________________________________________________________________
Train data pruned randomly at 40.0% :


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

{'Logistic Regression': {'Accuracy': 0.904, 'Precision': 0.023255813953488372, 'Recall': 0.14285714285714285, 'F1 Score': 0.04, 'ROC AUC': 0.7174731961750217, 'Confusion Matrix': array([[451,  42],
       [  6,   1]])}, 'SVM': {'Accuracy': 0.986, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.508838017965807, 'Confusion Matrix': array([[493,   0],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.926, 'Precision': 0.10526315789473684, 'Recall': 0.5714285714285714, 'F1 Score': 0.17777777777777778, 'ROC AUC': 0.751231527093596, 'Confusion Matrix': array([[459,  34],
       [  3,   4]])}}
_______________________________________________________________________________
Train data pruned randomly at 80.0% :


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


{'Logistic Regression': {'Accuracy': 0.856, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.65169516082295, 'Confusion Matrix': array([[428,  65],
       [  7,   0]])}, 'SVM': {'Accuracy': 0.91, 'Precision': 0.025, 'Recall': 0.14285714285714285, 'F1 Score': 0.0425531914893617, 'ROC AUC': 0.45696899449434947, 'Confusion Matrix': array([[454,  39],
       [  6,   1]])}, 'Decision Tree': {'Accuracy': 0.906, 'Precision': 0.08333333333333333, 'Recall': 0.5714285714285714, 'F1 Score': 0.14545454545454545, 'ROC AUC': 0.7410895392639816, 'Confusion Matrix': array([[449,  44],
       [  3,   4]])}}
_______________________________________________________________________________
Train data pruned randomly at 100.0% :


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


{'Logistic Regression': {'Accuracy': 0.538, 'Precision': 0.013043478260869565, 'Recall': 0.42857142857142855, 'F1 Score': 0.02531645569620253, 'ROC AUC': 0.5491161982034193, 'Confusion Matrix': array([[266, 227],
       [  4,   3]])}, 'SVM': {'Accuracy': 0.606, 'Precision': 0.010309278350515464, 'Recall': 0.2857142857142857, 'F1 Score': 0.01990049751243781, 'ROC AUC': 0.44537815126050423, 'Confusion Matrix': array([[301, 192],
       [  5,   2]])}, 'Decision Tree': {'Accuracy': 0.854, 'Precision': 0.05405405405405406, 'Recall': 0.5714285714285714, 'F1 Score': 0.09876543209876544, 'ROC AUC': 0.7147203709069836, 'Confusion Matrix': array([[423,  70],
       [  3,   4]])}}
_______________________________________________________________________________


# Calling SDV-Oversampling

In [23]:
sd1, train_df = do_sdv(X_train.drop(columns=['state']), y_train)
results_syn_sdv = dict()

# Add synthetic data at different percentages to the main DataFrame
for ratio in ratios:
    combined_df = add_synthetic_data(train_df, sd1, ratio)
    y_train_sdv = combined_df.iloc[:, -1]
    X_train_sdv = combined_df.iloc[:, :-1]

    preprocessed_X_train_sdv, scaler, imputer = preprocess_data_train(X_train_sdv)
    preprocessed_X_test_sdv = preprocess_data_test(X_test.drop(columns=['state']), scaler, imputer)

    X_train_sdv, y_train_sdv = preprocessed_X_train_sdv, y_train_sdv.to_numpy()
    X_test_sdv, y_test_sdv = preprocessed_X_test_sdv, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class:")
    results = evaluate_models(X_train_sdv, X_test_sdv, y_train_sdv, y_test_sdv)
    results_syn_sdv[ratio] = results
    print(results)
    print("_______________________________________________________________________________")

{
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1",
    "columns": {
        "id": {
            "sdtype": "id"
        },
        "security": {
            "sdtype": "numerical"
        },
        "wizard": {
            "sdtype": "categorical"
        },
        "service": {
            "sdtype": "categorical"
        },
        "secure": {
            "sdtype": "categorical"
        },
        "fails": {
            "sdtype": "numerical"
        },
        "permissions": {
            "sdtype": "numerical"
        },
        "allow": {
            "sdtype": "numerical"
        },
        "start": {
            "sdtype": "numerical"
        },
        "validation": {
            "sdtype": "numerical"
        },
        "user": {
            "sdtype": "numerical"
        },
        "cluster": {
            "sdtype": "numerical"
        },
        "cannot": {
            "sdtype": "categorical"
        },
        "set": {
            "sdtype": "categorical"
        },
        "make": {
 

/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:80: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


Train data combined with 20.0% synthetic data of minority class:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Logistic Regression': {'Accuracy': 0.944, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.5160822949869602, 'Confusion Matrix': array([[472,  21],
       [  7,   0]])}, 'SVM': {'Accuracy': 0.986, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.42683280208635177, 'Confusion Matrix': array([[493,   0],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.968, 'Precision': 0.09090909090909091, 'Recall': 0.14285714285714285, 'F1 Score': 0.1111111111111111, 'ROC AUC': 0.5612865835989568, 'Confusion Matrix': array([[483,  10],
       [  6,   1]])}}
_______________________________________________________________________________
Train data combined with 40.0% synthetic data of minority class:


...

{'Logistic Regression': {'Accuracy': 0.948, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.5024630541871922, 'Confusion Matrix': array([[474,  19],
       [  7,   0]])}, 'SVM': {'Accuracy': 0.986, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.4697189220515793, 'Confusion Matrix': array([[493,   0],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.972, 'Precision': 0.1111111111111111, 'Recall': 0.14285714285714285, 'F1 Score': 0.125, 'ROC AUC': 0.5633149811648797, 'Confusion Matrix': array([[485,   8],
       [  6,   1]])}}
_______________________________________________________________________________
Train data combined with 80.0% synthetic data of minority class:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Logistic Regression': {'Accuracy': 0.948, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.49492900608519264, 'Confusion Matrix': array([[474,  19],
       [  7,   0]])}, 'SVM': {'Accuracy': 0.986, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.482468849608809, 'Confusion Matrix': array([[493,   0],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.972, 'Precision': 0.1111111111111111, 'Recall': 0.14285714285714285, 'F1 Score': 0.125, 'ROC AUC': 0.5633149811648797, 'Confusion Matrix': array([[485,   8],
       [  6,   1]])}}
_______________________________________________________________________________
Train data combined with 100.0% synthetic data of minority class:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Logistic Regression': {'Accuracy': 0.948, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.5004346566212692, 'Confusion Matrix': array([[474,  19],
       [  7,   0]])}, 'SVM': {'Accuracy': 0.986, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.4914517531150391, 'Confusion Matrix': array([[493,   0],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.972, 'Precision': 0.1111111111111111, 'Recall': 0.14285714285714285, 'F1 Score': 0.125, 'ROC AUC': 0.5633149811648797, 'Confusion Matrix': array([[485,   8],
       [  6,   1]])}}
_______________________________________________________________________________


# Calling SMOTE-Oversampling

In [24]:
results_smote = dict()

for ratio in ratios:

    X_train_smote, y_train_smote = smote_oversampling(X_train.to_numpy(), y_train.to_numpy(), [ratio])
    preprocessed_X_train_smote, scaler, imputer = preprocess_data_train((np.array(X_train_smote))[0])
    preprocessed_X_test_smote = preprocess_data_test(X_test, scaler, imputer)

    X_train_smote, y_train_smote = preprocessed_X_train_smote, (np.array(y_train_smote))[0]
    X_test_smote, y_test_smote = preprocessed_X_test_smote, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class:")
    print(len(X_train_smote), len(y_train_smote))
    results = evaluate_models(X_train_smote, X_test_smote, y_train_smote, y_test_smote)
    results_smote[ratio] = results
    print(results)
    print("_______________________________________________________________________________")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data combined with 20.0% synthetic data of minority class:
591 591
{'Logistic Regression': {'Accuracy': 0.714, 'Precision': 0.014285714285714285, 'Recall': 0.2857142857142857, 'F1 Score': 0.027210884353741496, 'ROC AUC': 0.6374963778614893, 'Confusion Matrix': array([[355, 138],
       [  5,   2]])}, 'SVM': {'Accuracy': 0.978, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.6308316430020284, 'Confusion Matrix': array([[489,   4],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.956, 'Precision': 0.17391304347826086, 'Recall': 0.5714285714285714, 'F1 Score': 0.26666666666666666, 'ROC AUC': 0.766444508838018, 'Confusion Matrix': array([[474,  19],
       [  3,   4]])}}
_______________________________________________________________________________
Train data combined with 40.0% synthetic data of minority class:
682 682


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


{'Logistic Regression': {'Accuracy': 0.672, 'Precision': 0.024242424242424242, 'Recall': 0.5714285714285714, 'F1 Score': 0.046511627906976744, 'ROC AUC': 0.651984931903796, 'Confusion Matrix': array([[332, 161],
       [  3,   4]])}, 'SVM': {'Accuracy': 0.816, 'Precision': 0.03296703296703297, 'Recall': 0.42857142857142855, 'F1 Score': 0.061224489795918366, 'ROC AUC': 0.6528542451463344, 'Confusion Matrix': array([[405,  88],
       [  4,   3]])}, 'Decision Tree': {'Accuracy': 0.964, 'Precision': 0.21052631578947367, 'Recall': 0.5714285714285714, 'F1 Score': 0.3076923076923077, 'ROC AUC': 0.7705013039698637, 'Confusion Matrix': array([[478,  15],
       [  3,   4]])}}
_______________________________________________________________________________
Train data combined with 60.00000000000001% synthetic data of minority class:
773 773


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


{'Logistic Regression': {'Accuracy': 0.668, 'Precision': 0.01818181818181818, 'Recall': 0.42857142857142855, 'F1 Score': 0.03488372093023256, 'ROC AUC': 0.6528542451463344, 'Confusion Matrix': array([[331, 162],
       [  4,   3]])}, 'SVM': {'Accuracy': 0.628, 'Precision': 0.016216216216216217, 'Recall': 0.42857142857142855, 'F1 Score': 0.03125, 'ROC AUC': 0.6035931614024921, 'Confusion Matrix': array([[311, 182],
       [  4,   3]])}, 'Decision Tree': {'Accuracy': 0.932, 'Precision': 0.11428571428571428, 'Recall': 0.5714285714285714, 'F1 Score': 0.19047619047619044, 'ROC AUC': 0.7542741234424805, 'Confusion Matrix': array([[462,  31],
       [  3,   4]])}}
_______________________________________________________________________________
Train data combined with 80.0% synthetic data of minority class:
864 864


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


{'Logistic Regression': {'Accuracy': 0.668, 'Precision': 0.023952095808383235, 'Recall': 0.5714285714285714, 'F1 Score': 0.04597701149425287, 'ROC AUC': 0.6792234135033324, 'Confusion Matrix': array([[330, 163],
       [  3,   4]])}, 'SVM': {'Accuracy': 0.604, 'Precision': 0.015228426395939087, 'Recall': 0.42857142857142855, 'F1 Score': 0.029411764705882356, 'ROC AUC': 0.596638655462185, 'Confusion Matrix': array([[299, 194],
       [  4,   3]])}, 'Decision Tree': {'Accuracy': 0.966, 'Precision': 0.2222222222222222, 'Recall': 0.5714285714285714, 'F1 Score': 0.32, 'ROC AUC': 0.7715155027528253, 'Confusion Matrix': array([[479,  14],
       [  3,   4]])}}
_______________________________________________________________________________
Train data combined with 100.0% synthetic data of minority class:
956 956


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


{'Logistic Regression': {'Accuracy': 0.648, 'Precision': 0.022598870056497175, 'Recall': 0.5714285714285714, 'F1 Score': 0.043478260869565216, 'ROC AUC': 0.6337293538104898, 'Confusion Matrix': array([[320, 173],
       [  3,   4]])}, 'SVM': {'Accuracy': 0.616, 'Precision': 0.015706806282722512, 'Recall': 0.42857142857142855, 'F1 Score': 0.0303030303030303, 'ROC AUC': 0.5804114749348015, 'Confusion Matrix': array([[305, 188],
       [  4,   3]])}, 'Decision Tree': {'Accuracy': 0.974, 'Precision': 0.2857142857142857, 'Recall': 0.5714285714285714, 'F1 Score': 0.38095238095238093, 'ROC AUC': 0.7755722978846711, 'Confusion Matrix': array([[483,  10],
       [  3,   4]])}}
_______________________________________________________________________________


# Calling Random-Oversampling

In [25]:
results_random = dict()

for ratio in ratios:

    X_train_random, y_train_random = random_oversampling(X_train.to_numpy(), y_train.to_numpy(), [ratio])

    preprocessed_X_train_random, scaler, imputer = preprocess_data_train((np.array(X_train_random)[0]))
    preprocessed_X_test_random = preprocess_data_test(X_test, scaler, imputer)

    X_train_random, y_train_random = preprocessed_X_train_random, (np.array(y_train_random))[0]
    X_test_random, y_test_random = preprocessed_X_test_random, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class:")
    print(len(X_train_random), len(y_train_random))
    results = evaluate_models(X_train_random, X_test_random, y_train_random, y_test_random)
    results_random[ratio] = results
    print(results)
    print("_______________________________________________________________________________")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data combined with 20.0% synthetic data of minority class:
591 591
{'Logistic Regression': {'Accuracy': 0.936, 'Precision': 0.037037037037037035, 'Recall': 0.14285714285714285, 'F1 Score': 0.058823529411764705, 'ROC AUC': 0.6006954505940307, 'Confusion Matrix': array([[467,  26],
       [  6,   1]])}, 'SVM': {'Accuracy': 0.984, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.47551434366850187, 'Confusion Matrix': array([[492,   1],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.946, 'Precision': 0.14285714285714285, 'Recall': 0.5714285714285714, 'F1 Score': 0.2285714285714286, 'ROC AUC': 0.7613735149232107, 'Confusion Matrix': array([[469,  24],
       [  3,   4]])}}
_______________________________________________________________________________
Train data combined with 40.0% synthetic data of minority class:
682 682


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


{'Logistic Regression': {'Accuracy': 0.932, 'Precision': 0.034482758620689655, 'Recall': 0.14285714285714285, 'F1 Score': 0.05555555555555555, 'ROC AUC': 0.6024340770791075, 'Confusion Matrix': array([[465,  28],
       [  6,   1]])}, 'SVM': {'Accuracy': 0.982, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.4746450304259635, 'Confusion Matrix': array([[491,   2],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.954, 'Precision': 0.16666666666666666, 'Recall': 0.5714285714285714, 'F1 Score': 0.25806451612903225, 'ROC AUC': 0.7654303100550565, 'Confusion Matrix': array([[473,  20],
       [  3,   4]])}}
_______________________________________________________________________________
Train data combined with 60.00000000000001% synthetic data of minority class:
773 773


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


{'Logistic Regression': {'Accuracy': 0.93, 'Precision': 0.03333333333333333, 'Recall': 0.14285714285714285, 'F1 Score': 0.05405405405405406, 'ROC AUC': 0.6073601854534917, 'Confusion Matrix': array([[464,  29],
       [  6,   1]])}, 'SVM': {'Accuracy': 0.982, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.4804404520428861, 'Confusion Matrix': array([[491,   2],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.956, 'Precision': 0.17391304347826086, 'Recall': 0.5714285714285714, 'F1 Score': 0.26666666666666666, 'ROC AUC': 0.766444508838018, 'Confusion Matrix': array([[474,  19],
       [  3,   4]])}}
_______________________________________________________________________________
Train data combined with 80.0% synthetic data of minority class:
864 864


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


{'Logistic Regression': {'Accuracy': 0.926, 'Precision': 0.03125, 'Recall': 0.14285714285714285, 'F1 Score': 0.05128205128205128, 'ROC AUC': 0.598956824108954, 'Confusion Matrix': array([[462,  31],
       [  6,   1]])}, 'SVM': {'Accuracy': 0.982, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.4986960301361924, 'Confusion Matrix': array([[491,   2],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.956, 'Precision': 0.17391304347826086, 'Recall': 0.5714285714285714, 'F1 Score': 0.26666666666666666, 'ROC AUC': 0.766444508838018, 'Confusion Matrix': array([[474,  19],
       [  3,   4]])}}
_______________________________________________________________________________
Train data combined with 100.0% synthetic data of minority class:
956 956


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


{'Logistic Regression': {'Accuracy': 0.918, 'Precision': 0.027777777777777776, 'Recall': 0.14285714285714285, 'F1 Score': 0.046511627906976744, 'ROC AUC': 0.5882352941176471, 'Confusion Matrix': array([[458,  35],
       [  6,   1]])}, 'SVM': {'Accuracy': 0.98, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.486815415821501, 'Confusion Matrix': array([[490,   3],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.956, 'Precision': 0.17391304347826086, 'Recall': 0.5714285714285714, 'F1 Score': 0.26666666666666666, 'ROC AUC': 0.766444508838018, 'Confusion Matrix': array([[474,  19],
       [  3,   4]])}}
_______________________________________________________________________________


# Calling SVM-SMOTE Over-Sampling

In [26]:
results_svm_smote = dict()

for ratio in ratios:

    X_train_svm_smote, y_train_svm_smote = svm_smote_oversampling(X_train.to_numpy(), y_train.to_numpy(), [ratio])

    preprocessed_X_train_svm_smote, scaler, imputer = preprocess_data_train((np.array(X_train_svm_smote))[0])
    preprocessed_X_test_svm_smote = preprocess_data_test(X_test, scaler, imputer)

    X_train_svm_smote, y_train_svm_smote = preprocessed_X_train_svm_smote, (np.array(y_train_svm_smote))[0]
    X_test_svm_smote, y_test_svm_smote = preprocessed_X_test_svm_smote, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class:")
    print(len(X_train_svm_smote), len(y_train_svm_smote))
    results = evaluate_models(X_train_svm_smote, X_test_svm_smote, y_train_svm_smote, y_test_svm_smote)
    results_svm_smote[ratio] = results
    print(results)
    print("_______________________________________________________________________________")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data combined with 20.0% synthetic data of minority class:
550 550
{'Logistic Regression': {'Accuracy': 0.582, 'Precision': 0.02358490566037736, 'Recall': 0.7142857142857143, 'F1 Score': 0.04566210045662101, 'ROC AUC': 0.6627064618951028, 'Confusion Matrix': array([[286, 207],
       [  2,   5]])}, 'SVM': {'Accuracy': 0.982, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.6548826427122573, 'Confusion Matrix': array([[491,   2],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.968, 'Precision': 0.2, 'Recall': 0.42857142857142855, 'F1 Score': 0.27272727272727276, 'ROC AUC': 0.7021153288901768, 'Confusion Matrix': array([[481,  12],
       [  4,   3]])}}
_______________________________________________________________________________
Train data combined with 40.0% synthetic data of minority class:
600 600


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


{'Logistic Regression': {'Accuracy': 0.544, 'Precision': 0.021645021645021644, 'Recall': 0.7142857142857143, 'F1 Score': 0.04201680672268907, 'ROC AUC': 0.632280498406259, 'Confusion Matrix': array([[267, 226],
       [  2,   5]])}, 'SVM': {'Accuracy': 0.928, 'Precision': 0.08571428571428572, 'Recall': 0.42857142857142855, 'F1 Score': 0.14285714285714285, 'ROC AUC': 0.6841495218777165, 'Confusion Matrix': array([[461,  32],
       [  4,   3]])}, 'Decision Tree': {'Accuracy': 0.97, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.4918864097363083, 'Confusion Matrix': array([[485,   8],
       [  7,   0]])}}
_______________________________________________________________________________
Train data combined with 60.00000000000001% synthetic data of minority class:
650 650


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


{'Logistic Regression': {'Accuracy': 0.544, 'Precision': 0.021645021645021644, 'Recall': 0.7142857142857143, 'F1 Score': 0.04201680672268907, 'ROC AUC': 0.6409736308316429, 'Confusion Matrix': array([[267, 226],
       [  2,   5]])}, 'SVM': {'Accuracy': 0.792, 'Precision': 0.02912621359223301, 'Recall': 0.42857142857142855, 'F1 Score': 0.05454545454545455, 'ROC AUC': 0.6731382208055636, 'Confusion Matrix': array([[393, 100],
       [  4,   3]])}, 'Decision Tree': {'Accuracy': 0.958, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.48580121703853957, 'Confusion Matrix': array([[479,  14],
       [  7,   0]])}}
_______________________________________________________________________________
Train data combined with 80.0% synthetic data of minority class:
700 700


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


{'Logistic Regression': {'Accuracy': 0.536, 'Precision': 0.02127659574468085, 'Recall': 0.7142857142857143, 'F1 Score': 0.04132231404958678, 'ROC AUC': 0.6424224862358736, 'Confusion Matrix': array([[263, 230],
       [  2,   5]])}, 'SVM': {'Accuracy': 0.65, 'Precision': 0.022727272727272728, 'Recall': 0.5714285714285714, 'F1 Score': 0.04371584699453552, 'ROC AUC': 0.6363372935381049, 'Confusion Matrix': array([[321, 172],
       [  3,   4]])}, 'Decision Tree': {'Accuracy': 0.958, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.48580121703853957, 'Confusion Matrix': array([[479,  14],
       [  7,   0]])}}
_______________________________________________________________________________
Train data combined with 100.0% synthetic data of minority class:
751 751


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


{'Logistic Regression': {'Accuracy': 0.528, 'Precision': 0.02092050209205021, 'Recall': 0.7142857142857143, 'F1 Score': 0.04065040650406504, 'ROC AUC': 0.6557519559547956, 'Confusion Matrix': array([[259, 234],
       [  2,   5]])}, 'SVM': {'Accuracy': 0.624, 'Precision': 0.021164021164021163, 'Recall': 0.5714285714285714, 'F1 Score': 0.04081632653061224, 'ROC AUC': 0.6114169805853376, 'Confusion Matrix': array([[308, 185],
       [  3,   4]])}, 'Decision Tree': {'Accuracy': 0.964, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.48884381338742394, 'Confusion Matrix': array([[482,  11],
       [  7,   0]])}}
_______________________________________________________________________________


# No-Sampling Results

In [27]:
results_no_sampling = dict()

for ratio in ratios:

    X_train_no_sampling, y_train_no_sampling = X_train.to_numpy(), y_train.to_numpy()

    preprocessed_X_train_no_sampling, scaler, imputer = preprocess_data_train(X_train_no_sampling)
    preprocessed_X_test_no_sampling = preprocess_data_test(X_test, scaler, imputer)

    X_train_no_sampling, y_train_no_sampling = preprocessed_X_train_no_sampling, y_train_no_sampling
    X_test_no_sampling, y_test_no_sampling = preprocessed_X_test_no_sampling, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class:")
    print(len(X_train_no_sampling), len(y_train_no_sampling))
    results = evaluate_models(X_train_no_sampling, X_test_no_sampling, y_train_no_sampling, y_test_no_sampling)
    results_no_sampling[ratio] = results
    print(results)
    print("_______________________________________________________________________________")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data combined with 20.0% synthetic data of minority class:
500 500


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Logistic Regression': {'Accuracy': 0.952, 'Precision': 0.05263157894736842, 'Recall': 0.14285714285714285, 'F1 Score': 0.07692307692307693, 'ROC AUC': 0.6146044624746451, 'Confusion Matrix': array([[475,  18],
       [  6,   1]])}, 'SVM': {'Accuracy': 0.986, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.46276441611127206, 'Confusion Matrix': array([[493,   0],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.968, 'Precision': 0.09090909090909091, 'Recall': 0.14285714285714285, 'F1 Score': 0.1111111111111111, 'ROC AUC': 0.5612865835989568, 'Confusion Matrix': array([[483,  10],
       [  6,   1]])}}
_______________________________________________________________________________
Train data combined with 40.0% synthetic data of minority class:
500 500


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

{'Logistic Regression': {'Accuracy': 0.952, 'Precision': 0.05263157894736842, 'Recall': 0.14285714285714285, 'F1 Score': 0.07692307692307693, 'ROC AUC': 0.6146044624746451, 'Confusion Matrix': array([[475,  18],
       [  6,   1]])}, 'SVM': {'Accuracy': 0.986, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.46276441611127206, 'Confusion Matrix': array([[493,   0],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.968, 'Precision': 0.09090909090909091, 'Recall': 0.14285714285714285, 'F1 Score': 0.1111111111111111, 'ROC AUC': 0.5612865835989568, 'Confusion Matrix': array([[483,  10],
       [  6,   1]])}}
_______________________________________________________________________________
Train data combined with 80.0% synthetic data of minority class:
500 500


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Logistic Regression': {'Accuracy': 0.952, 'Precision': 0.05263157894736842, 'Recall': 0.14285714285714285, 'F1 Score': 0.07692307692307693, 'ROC AUC': 0.6146044624746451, 'Confusion Matrix': array([[475,  18],
       [  6,   1]])}, 'SVM': {'Accuracy': 0.986, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.46276441611127206, 'Confusion Matrix': array([[493,   0],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.968, 'Precision': 0.09090909090909091, 'Recall': 0.14285714285714285, 'F1 Score': 0.1111111111111111, 'ROC AUC': 0.5612865835989568, 'Confusion Matrix': array([[483,  10],
       [  6,   1]])}}
_______________________________________________________________________________
Train data combined with 100.0% synthetic data of minority class:
500 500


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Logistic Regression': {'Accuracy': 0.952, 'Precision': 0.05263157894736842, 'Recall': 0.14285714285714285, 'F1 Score': 0.07692307692307693, 'ROC AUC': 0.6146044624746451, 'Confusion Matrix': array([[475,  18],
       [  6,   1]])}, 'SVM': {'Accuracy': 0.986, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.46276441611127206, 'Confusion Matrix': array([[493,   0],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.968, 'Precision': 0.09090909090909091, 'Recall': 0.14285714285714285, 'F1 Score': 0.1111111111111111, 'ROC AUC': 0.5612865835989568, 'Confusion Matrix': array([[483,  10],
       [  6,   1]])}}
_______________________________________________________________________________
